In [1]:
# !pip install -r requirements.txt
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib
import re
import nltk
import numpy as np
from IPython.display import Image
import pickle
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('words')

In [2]:
professioni = pd.read_csv('data/professioni.csv')
dati_aprile = pd.read_csv('Job_vacancies_aprile_completo.csv', skiprows=1, usecols = [1,2,3],\
                          names = ['Target','Sub_Role','Job_Description'])
dati_aprile = dati_aprile.reindex(np.random.permutation(dati_aprile.index))
print('dati_aprile shape {}'.format(dati_aprile.shape))
dati_aprile[['Target','Job_Description']].head() # per facilitare la comprensione dei dati di testo da parte del modello

dati_aprile shape (650, 3)


,Target,Job_Description
26,Elettromeccanici,Il/la candidato/a ideale è in possesso di un d...
172,Cuochi in alberghi e ristoranti,"Gi Group SpA, Agenzia per il Lavoro (Aut. Min...."
359,Tecnici web,Unix-Linux System Administrator Candidati Desc...
43,Elettromeccanici,Il candidato risponderà direttamente alla Resp...
440,Camerieri di ristorante,"L’Hassler Roma, prestigioso Hotel 5 stelle Lus..."


In [3]:
# remove punctuation
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: re.sub('[^a-zA-Z]',' ',x))
dati_aprile.Target = dati_aprile.Target.apply(lambda x: x.lower()) #tutte le parole in minuscolo 
dati_aprile.Job_Description = dati_aprile['Job_Description'].apply(lambda x: word_tokenize(x.lower())) # lowercase each word
dati_aprile[['Target','Job_Description']].head()

,Target,Job_Description
26,elettromeccanici,"[il, la, candidato, a, ideale, in, possesso, d..."
172,cuochi in alberghi e ristoranti,"[gi, group, spa, agenzia, per, il, lavoro, aut..."
359,tecnici web,"[unix, linux, system, administrator, candidati..."
43,elettromeccanici,"[il, candidato, risponder, direttamente, alla,..."
440,camerieri di ristorante,"[l, hassler, roma, prestigioso, hotel, stelle,..."


In [4]:
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if len(i) >= 3])
dati_aprile.Job_Description.head() # note the word 'vimercate' in row = 0

26     [candidato, ideale, possesso, diploma, tipo, t...
172    [group, spa, agenzia, per, lavoro, aut, min, p...
359    [unix, linux, system, administrator, candidati...
43     [candidato, risponder, direttamente, alla, res...
440    [hassler, roma, prestigioso, hotel, stelle, lu...
Name: Job_Description, dtype: object

In [5]:
##### remove in 'job_description' words from 'fonte'
Fonti = ['randstad', 'monster', 'infojob', 'technical', 'kelly', 'services', 'italia', 'lavoropi',\
             'quanta','adeccp','spa','vimercate','temporary','openjobmetis','agenzia']
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if i not in Fonti])
dati_aprile.Job_Description.head(5) # compare row 0 and row 2 with previous row 0 and row 2

26     [candidato, ideale, possesso, diploma, tipo, t...
172    [group, per, lavoro, aut, min, prot, ricerca, ...
359    [unix, linux, system, administrator, candidati...
43     [candidato, risponder, direttamente, alla, res...
440    [hassler, roma, prestigioso, hotel, stelle, lu...
Name: Job_Description, dtype: object

In [7]:
list_stopwords = list(set(stopwords.words('italian')))
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if i not in list_stopwords] )
dati_aprile.Job_Description.head(5)

26     [candidato, ideale, possesso, diploma, tipo, t...
172    [group, lavoro, aut, min, prot, ricerca, prest...
359    [unix, linux, system, administrator, candidati...
43     [candidato, risponder, direttamente, responsab...
440    [hassler, roma, prestigioso, hotel, stelle, lu...
Name: Job_Description, dtype: object

In [8]:
# load a text file containing a list of 'Comuni, Provincie, Regioni' italiane
geo_data = pd.read_fwf('data/listacomuni.txt')
geo_data.to_csv('data/listacomuni.csv')
geo_data = pd.read_csv('data/listacomuni.csv', sep=';')
comune = [i.lower() for i in geo_data.Comune]
geo_data.Provincia = geo_data.Provincia.apply(lambda x: str(x))
provincia = [i.lower() for i in geo_data.Provincia]
regione = [i.lower() for i in geo_data.Regione] + ['lazio']

def remove_comuni(data, column, geo_data):
    ''' function to remove geographic noisy'''
    comune = [i.lower() for i in geo_data.Comune]
    geo_data.Provincia = geo_data.Provincia.apply(lambda x: str(x))
    provincia = [i.lower() for i in geo_data.Provincia]
    regione = [i.lower() for i in geo_data.Regione]  + ['lazio']
    
    data[column] = data[column].apply(lambda x: [i for i in x if i not in comune])
    data[column] = data[column].apply(lambda x: [i for i in x if i not in provincia])
    data[column] = data[column].apply(lambda x: [i for i in x if i not in regione])
    
    return data

In [9]:
dati_aprile = remove_comuni(dati_aprile, 'Job_Description', geo_data)
dati_aprile[['Target','Job_Description']].head()

,Target,Job_Description
26,elettromeccanici,"[candidato, ideale, possesso, diploma, tipo, t..."
172,cuochi in alberghi e ristoranti,"[group, lavoro, aut, min, prot, ricerca, prest..."
359,tecnici web,"[unix, linux, system, administrator, candidati..."
43,elettromeccanici,"[candidato, risponder, direttamente, responsab..."
440,camerieri di ristorante,"[hassler, prestigioso, hotel, stelle, lusso, c..."


In [10]:
italian_vocab = pd.read_fwf('data/660000_parole_italiane.txt', names = ['word'])
italian_vocab = set(italian_vocab.word.tolist())
english_vocab = sorted(set(w.lower() for w in nltk.corpus.words.words())) # english vocabulary

In [11]:
def uncommon_words(data,column, italian_vocab, english_vocab = None):
    data[column] = data[column].apply(lambda x: [i for i in x if i in italian_vocab])
    return data

In [12]:
dati_aprile = uncommon_words(dati_aprile, 'Job_Description', italian_vocab, english_vocab)
dati_aprile[['Target','Job_Description']].head()# note that the word 'vimercate' is not present anymore

,Target,Job_Description
26,elettromeccanici,"[candidato, ideale, possesso, diploma, tipo, t..."
172,cuochi in alberghi e ristoranti,"[lavoro, ricerca, prestigioso, boutique, hotel..."
359,tecnici web,"[candidati, descrizione, azienda, regolarmente..."
43,elettromeccanici,"[candidato, direttamente, responsabile, manute..."
440,camerieri di ristorante,"[prestigioso, hotel, stelle, lusso, centro, se..."


In [14]:
stemmer = nltk.stem.snowball.ItalianStemmer(ignore_stopwords=False)

def stemming(dataset):
    ''' stemming a text '''
    dataset['Job_Description'] = dataset['Job_Description'].apply(lambda x: [stemmer.stem(i) \
                                                                        for i in x])
    return dataset

dati_aprile = stemming(dati_aprile)
dati_aprile.head()

,Target,Sub_Role,Job_Description
26,elettromeccanici,Manutentore Elettromeccanico,"[candid, ideal, possess, diplom, tip, tecnic, ..."
172,cuochi in alberghi e ristoranti,Cuoco,"[lavor, ricerc, prestig, boutiqu, hotel, figur..."
359,tecnici web,Amministratore web,"[candid, descrizion, azi, regolar, autorizz, m..."
43,elettromeccanici,Manutentore Elettromeccanico,"[candid, dirett, respons, manutenzion, manuten..."
440,camerieri di ristorante,Cameriere/a di sala/ristorante extra,"[prestig, hotel, stell, luss, centr, selezion,..."


In [15]:
noisy_words = ['degli','tipi','del','dei','delle','per','nel','dell',\
               'sui','less','sul','che','jsss','teor','dal','della']
dati_aprile['Job_Description'] = dati_aprile['Job_Description'].apply(lambda x: [i for i in x if i not in noisy_words])
dati_aprile['Job_Description'] .head()

26     [candid, ideal, possess, diplom, tip, tecnic, ...
172    [lavor, ricerc, prestig, boutiqu, hotel, figur...
359    [candid, descrizion, azi, regolar, autorizz, m...
43     [candid, dirett, respons, manutenzion, manuten...
440    [prestig, hotel, stell, luss, centr, selezion,...
Name: Job_Description, dtype: object

In [17]:
Role_dictionary = pd.Series(dati_aprile['Target'].unique()).to_dict()
Role_dictionary = dict([(value, key) for key, value in Role_dictionary.items()])
with open("Role_dictionary.pkl", 'wb') as handle:
                pickle.dump(Role_dictionary, handle) 
Role_dictionary

{'elettromeccanici': 0,
 'cuochi in alberghi e ristoranti': 1,
 'tecnici web': 2,
 'camerieri di ristorante': 3,
 'statistici': 4,
 'commessi delle vendite al minuto': 5}

In [18]:
dati_aprile['Multi_Class'] = dati_aprile['Target'].apply(lambda x: Role_dictionary.get(x,'Unknown'))
dati_aprile[['Target','Multi_Class','Job_Description']]

,Target,Multi_Class,Job_Description
26,elettromeccanici,0,"[candid, ideal, possess, diplom, tip, tecnic, ..."
172,cuochi in alberghi e ristoranti,1,"[lavor, ricerc, prestig, boutiqu, hotel, figur..."
359,tecnici web,2,"[candid, descrizion, azi, regolar, autorizz, m..."
43,elettromeccanici,0,"[candid, dirett, respons, manutenzion, manuten..."
440,camerieri di ristorante,3,"[prestig, hotel, stell, luss, centr, selezion,..."
...,...,...,...
580,commessi delle vendite al minuto,5,"[import, italian, oper, vend, accessor, ricamb..."
640,commessi delle vendite al minuto,5,"[addett, addett, vend, showroom, risors, ricer..."
377,tecnici web,2,"[marketing, gestion, web, sit, event, azi, ost..."
271,statistici,4,"[lavor, filial, ricerc, client, oper, settor, ..."


In [19]:
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: ' '.join(x))
dati_aprile.Job_Description.head()

26     candid ideal possess diplom tip tecnic matur e...
172    lavor ricerc prestig boutiqu hotel figur cuoc ...
359    candid descrizion azi regolar autorizz ministe...
43     candid dirett respons manutenzion manutenzion ...
440    prestig hotel stell luss centr selezion cameri...
Name: Job_Description, dtype: object

In [20]:
X_train, X_test, y_train, y_test = train_test_split(dati_aprile['Job_Description'], 
                                                    dati_aprile['Multi_Class'],
                                                   test_size=0.20, random_state=42)

print('Number of rows in the total set: {}'.format(dati_aprile.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 650
Number of rows in the training set: 520
Number of rows in the test set: 130


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
count_vector.fit(X_train)
with open("count_vector2104.pkl", 'wb') as handle:
                    pickle.dump(count_vector, handle)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [22]:
# Fit the training data and then return the matrix
training_data = count_vector.transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

In [23]:
print('training data',training_data.shape)
print('testing data',testing_data.shape)

training data (520, 2455)
testing data (130, 2455)


In [24]:
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)
import joblib
joblib.dump(naive_bayes, 'model2104.sav')

['model2104.sav']

In [25]:
def get_key(value, dictionary):
    for k,v in dictionary.items():
        if value == v:
            return k

In [26]:
predictions = naive_bayes.predict(testing_data)
predictions_keys = []
for i in predictions:
    predictions_keys.append(get_key(i,Role_dictionary))

actual_predictions = []
for i in y_test.tolist():
    actual_predictions.append(get_key(i,Role_dictionary))

In [27]:
# check the accuracy of our model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test,predictions)))

Accuracy score:  0.9461538461538461
